# **CHIRPS DATA EXTRACTION**

In [1]:
import ee
import geemap
import folium
import pandas as pd
import numpy as np
import datetime
import matplotlib.pyplot as plt

In [ ]:
ee.Authenticate()


In [ ]:
ee.Initialize(project= "ee-chirps-valid-em")

In [ ]:
# Ruta al FeatureCollection en GEE
feature_collection_path = 'projects/ee-chirps-valid-em/assets/shp_em'
# Cargar el FeatureCollection
points = ee.FeatureCollection(feature_collection_path)
# Crear un mapa centrado en una ubicación específica
Map = geemap.Map(center=[20, -100], zoom=5)
# Añadir la nube de puntos al mapa
Map.addLayer(points, {}, 'Puntos de muestreo')
# Mostrar el mapa
Map

Map(center=[20, -100], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(ch…

In [ ]:
# Cargar el FeatureCollection
points = ee.FeatureCollection(feature_collection_path)
# Convertir el FeatureCollection a una lista de diccionarios
attributes = points.getInfo()['features']
# Crear una lista de diccionarios con las propiedades
properties = [feature['properties'] for feature in attributes]
# Convertir la lista de diccionarios a un DataFrame de pandas
df = pd.DataFrame(properties)
# Mostrar la tabla
df

,altitud,estacion,estado,fecha_fina,fecha_inic,latitud,longitud,municipio,nombre,organismo,situacion
0,300 msnm,12061,GUERRERO,2021-05-31,1961-08-25,16.684,-98.402,Ometepec,OMETEPEC (CFE),CFE,1
1,1645 msnm,15046,MEXICO,2021-07-31,1949-07-01,19.176,-100.216,Valle de Bravo,PRESA COLORINES (CFE),CFE,1
2,1850 msnm,15130,MEXICO,2021-07-31,1969-01-01,19.204,-100.181,Valle de Bravo,PRESA VALLE DEL BRAVO (CFE),CFE,1
3,1110 msnm,12091,GUERRERO,2021-10-31,1961-01-01,17.540,-98.591,Tlapa de Comonfort,TLAPA (CFE),CFE,1
4,2098 msnm,21021,PUEBLA,2022-12-31,1954-11-12,19.792,-97.761,TETELA DE OCAMPO,CAPULUAQUE (CFE),CFE,1
...,...,...,...,...,...,...,...,...,...,...,...
1196,360 msnm,6013,COLIMA,2021-12-31,1960-06-12,19.056,-103.786,Tecom�n,LA SALADA,CONAGUA-DGE,1
1197,195 msnm,6017,COLIMA,2021-12-31,1970-03-01,19.112,-103.884,Tecom�n,MADRID,CONAGUA-DGE,1
1198,30 msnm,6058,COLIMA,2021-12-31,1963-01-01,18.908,-103.874,Tecom�n,TECOMAN (DGE),CONAGUA-DGE,1
1199,48 msnm,6059,COLIMA,2021-12-31,1991-09-12,18.902,-103.792,Tecom�n,LAGUNA DE ALCUZAHUE,CONAGUA-DGE,1


In [ ]:
# Definir el conjunto de datos CHIRPS diarios
chirps = ee.ImageCollection("UCSB-CHG/CHIRPS/DAILY")

# Cargar la colección de estaciones y filtrar las primeras 5
estaciones = ee.FeatureCollection("projects/ee-chirps-valid-em/assets/shp_em").limit(5)

# Definir el período de tiempo
startDate = '2000-01-01'
endDate = '2020-12-31'

# Filtrar los datos CHIRPS por el período de tiempo
chirps_filtered = chirps.filterDate(startDate, endDate)

# Función para rellenar con ceros a la izquierda hasta 5 dígitos
def pad_left(num, size):
    s = str(num)
    while len(s) < size:
        s = "0" + s
    return s

# Generar tareas de exportación
tasks = []

def export_estacion(estacion):
    try:
        estacion_id = pad_left(estacion.get('estacion').getInfo(), 5)

        def get_precipitation(image):
            precipitation = image.reduceRegion(
                reducer=ee.Reducer.first(),
                geometry=estacion.geometry(),
                scale=5000,
                maxPixels=1e9
            ).get('precipitation')
            return ee.Feature(None, {
                'date': image.date().format('YYYY-MM-dd'),
                'precipitation': precipitation
            })

        chirps_estacion = chirps_filtered.map(get_precipitation)

        export_data = chirps_estacion.map(lambda feature: ee.Feature(None, {
            'Fecha': ee.Date(feature.get('date')).format('dd/MM/yyyy'),
            'Precipitación (mm)': feature.get('precipitation')
        }))

        task = ee.batch.Export.table.toDrive(
            collection=export_data,
            description='PrecipEst_' + estacion_id,
            fileNamePrefix=estacion_id,
            fileFormat='CSV',
            folder='prueba_1',
            selectors=['Fecha', 'Precipitación (mm)']
        )
        tasks.append(task)
    except Exception as e:
        print(f"Error processing estacion {estacion.getInfo()}: {e}")

# Evaluar la lista de estaciones y ejecutar la exportación para cada una
estaciones_list = estaciones.getInfo()['features']
for estacion_dict in estaciones_list:
    estacion = ee.Feature(estacion_dict)
    export_estacion(estacion)

# Ejecutar las tareas
for task in tasks:
    task.start()
    print(f"Started task for estacion {task.config['description']}")
    time.sleep(1)  # Agregar un pequeño retraso entre tareas para evitar sobrecarga

print("All tasks have been started.")


Error processing estacion {'type': 'Feature', 'geometry': {'type': 'Point', 'coordinates': [-98.40200175780339, 16.683998596632012]}, 'id': '00000000000000000169', 'properties': {'altitud': '300 msnm', 'estacion': 12061, 'estado': 'GUERRERO', 'fecha_fina': '2021-05-31', 'fecha_inic': '1961-08-25', 'latitud': 16.684, 'longitud': -98.402, 'municipio': 'Ometepec', 'nombre': 'OMETEPEC (CFE)', 'organismo': 'CFE', 'situacion': 1}}: pad_left() missing 1 required positional argument: 'size'
Error processing estacion {'type': 'Feature', 'geometry': {'type': 'Point', 'coordinates': [-100.2159997391536, 19.175998434057618]}, 'id': '0000000000000000023e', 'properties': {'altitud': '1645 msnm', 'estacion': 15046, 'estado': 'MEXICO', 'fecha_fina': '2021-07-31', 'fecha_inic': '1949-07-01', 'latitud': 19.176, 'longitud': -100.216, 'municipio': 'Valle de Bravo', 'nombre': 'PRESA COLORINES (CFE)', 'organismo': 'CFE', 'situacion': 1}}: pad_left() missing 1 required positional argument: 'size'
Error proce

KeyboardInterrupt: 

In [ ]:

# Definir el conjunto de datos CHIRPS diarios
chirps = ee.ImageCollection("UCSB-CHG/CHIRPS/DAILY")

# Cargar la colección de estaciones
estaciones = ee.FeatureCollection("projects/ee-chirps-valid-em/assets/shp_em")

# Definir el período de tiempo
startDate = '2000-01-01'
endDate = '2020-12-31'

# Filtrar los datos CHIRPS por el período de tiempo
chirps_filtered = chirps.filterDate(startDate, endDate)

# Función para rellenar con ceros a la izquierda hasta 5 dígitos
def pad_left(num, size):
    s = str(num)
    while len(s) < size:
        s = "0" + s
    return s

# Función para exportar datos de precipitación para un lote de estaciones
def export_batch(estaciones_batch):
    tasks = []

    def export_estacion(estacion):
        try:
            estacion_id = pad_left(estacion.get('estacion').getInfo(), 5)

            def get_precipitation(image):
                precipitation = image.reduceRegion(
                    reducer=ee.Reducer.first(),
                    geometry=estacion.geometry(),
                    scale=5000,
                    maxPixels=1e9
                ).get('precipitation')
                return ee.Feature(None, {
                    'date': image.date().format('YYYY-MM-dd'),
                    'precipitation': precipitation
                })

            chirps_estacion = chirps_filtered.map(get_precipitation)

            export_data = chirps_estacion.map(lambda feature: ee.Feature(None, {
                'Fecha': ee.Date(feature.get('date')).format('dd/MM/yyyy'),
                'Precipitación (mm)': feature.get('precipitation')
            }))

            task = ee.batch.Export.table.toDrive(
                collection=export_data,
                description='PrecipEst_' + estacion_id,
                fileNamePrefix=estacion_id,
                fileFormat='CSV',
                folder='prueba_2',
                selectors=['Fecha', 'Precipitación (mm)']
            )
            tasks.append(task)
        except Exception as e:
            print(f"Error processing estacion {estacion.getInfo()}: {e}")

    for estacion_dict in estaciones_batch:
        estacion = ee.Feature(estacion_dict)
        export_estacion(estacion)

    for task in tasks:
        task.start()
        print(f"Started task for estacion {task.config['description']}")
        time.sleep(1)  # Agregar un pequeño retraso entre tareas para evitar sobrecarga

# Dividir el conjunto de estaciones en lotes de 100 estaciones
batch_size = 15
estaciones_list = estaciones.getInfo()['features']
num_batches = (len(estaciones_list) + batch_size - 1) // batch_size

for i in range(num_batches):
    batch_start = i * batch_size
    batch_end = min((i + 1) * batch_size, len(estaciones_list))
    estaciones_batch = estaciones_list[batch_start:batch_end]
    print(f"Processing batch {i + 1} of {num_batches}")
    export_batch(estaciones_batch)

print("All tasks have been started.")


Processing batch 1 of 81
Started task for estacion PrecipEst_12061
Started task for estacion PrecipEst_15046
Started task for estacion PrecipEst_15130
Started task for estacion PrecipEst_12091
Started task for estacion PrecipEst_21021
Started task for estacion PrecipEst_21047
Started task for estacion PrecipEst_04078
Started task for estacion PrecipEst_04082
Started task for estacion PrecipEst_04085
Started task for estacion PrecipEst_12128
Started task for estacion PrecipEst_19152
Started task for estacion PrecipEst_19189
Started task for estacion PrecipEst_23159
Started task for estacion PrecipEst_23160
Started task for estacion PrecipEst_23162
Processing batch 2 of 81
Started task for estacion PrecipEst_23167
Started task for estacion PrecipEst_30457
Started task for estacion PrecipEst_30462
Started task for estacion PrecipEst_31097
Started task for estacion PrecipEst_15137
Started task for estacion PrecipEst_10170
Started task for estacion PrecipEst_10169
Started task for estacion 